In [74]:
# Source config and checkpoints
#checkpoint_path = "gs://patemotter-maxtext-out/patemotter-1b-maxtext-model/checkpoints/3399"
#config_path = "gs://patemotter-maxtext-out/patemotter-1b-maxtext-model/base.yml"

# Copied files into our own bucket
checkpoint_path = "gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints/3399"
config_path = "gs://patflick-maxtext-lingvo/patemotter-1b/base.yml"
config_path = "config/base-1b.yml"

In [75]:
!hostname
!pwd

t1v-n-d84a2c81-w-0
/home/patflick/dev/git/maxtext/MaxText


In [76]:
my_bucket = "gs://patflick-maxtext-lingvo"
base_output_directory= my_bucket + "/maxtext/1b/2023114"

checkpoint_path = "gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints"
extra_config = {
    # Run config
    "run_name": "finetuning-1b-v1",
    "base_output_directory": base_output_directory,

    # Load checkpoint
    "load_from_other_directory": checkpoint_path,
    "load_from_other_directory_step": 3399,
   # "file_pattern_for_train_data": "gs://unused", # TODO: fix upstream

    # Parallelism
    "dcn_tensor_parallelism": 4,
    "ici_tensor_parallelism": 4,

    # c4 dataset downloaded via `download_dataset.sh` from gs://allennlp-tensorflow-datasets/c4/en/3.0.1/
    "dataset_path": my_bucket + "/",
    "assets_path": "/home/patflick/dev/git/maxtext/assets", # TODO: move assets to GCS

    # Config for `decode_loop`
    "prompt": "I like ",
    "steps": 10,
    "max_predict_length": 50,
}

import pyconfig
pyconfig.initialize(["", "configs/base-1b.yml"] + [f"{k}={v}" for k,v in extra_config.items()])
config = pyconfig.config
pyconfig._config.keys

OrderedDict([('run_name', 'finetuning-1b-v1'),
             ('load_parameters_path', ''),
             ('load_from_other_directory',
              'gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints'),
             ('load_from_other_directory_step', 3399),
             ('reuse_example_batch', 0),
             ('metrics_file', ''),
             ('gcs_metrics', False),
             ('dtype', dtype(bfloat16)),
             ('int8_training', False),
             ('use_dqdg', False),
             ('fwd_int8', True),
             ('dlhs_int8', True),
             ('drhs_int8', False),
             ('fwd_int8_qk', False),
             ('dlhs_int8_qk', False),
             ('drhs_int8_qk', False),
             ('fwd_int8_pv', True),
             ('dlhs_int8_pv', True),
             ('drhs_int8_pv', False),
             ('aqt_use_dummy_static_bound', False),
             ('aqt_use_fwd_quant', False),
             ('aqt_rng_type', 'jax.uniform'),
             ('global_parameter_scale', 1),
 

In [77]:
# MANUALLY load the training data and tokenizer
import max_utils  
from jax.sharding import Mesh
from input_pipeline import create_data_iterator_with_tokenizer

# Mesh definition
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)


train_ds, sp_tokenizer = create_data_iterator_with_tokenizer(config, mesh)


Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer


In [78]:
import tensorflow_datasets as tfds
from input_pipeline import get_datasets, preprocess_dataset
import os

read_config = tfds.ReadConfig(
    shuffle_seed = config.data_shuffle_seed,
)
train_ds, eval_ds = get_datasets(
    config=config,
    read_config = read_config,
)
train_iter, eval_iter, predict_iter, sp_tokenizer = preprocess_dataset(
    config,
    mesh,
    train_ds, eval_ds,
    vocab_path=os.path.join(config.assets_path, config.vocab_relative_path),
    data_shuffle_seed = config.data_shuffle_seed,
)



Model path: /home/patflick/dev/git/maxtext/assets/tokenizer


In [83]:
!pip install seqio t5

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.5/164.5 KB 4.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 40.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 80.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 116.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.2/385.2 KB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.4/353.4 KB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 76.0 MB/s eta 0:00:00
  Preparing metadata (setup.py)

In [ ]:
# attempt to load cnn_dailymail dataset using qinwen's loader

import cnndm_data


# TODO: we need a new config!
cnndm_data.make_cnndm_train_iterator_and_tokenizer(config)

In [58]:
# load model
from layers import Transformer
model = Transformer(config, mesh)

In [59]:
model # un-initialized

Transformer(
    # attributes
    config = <pyconfig.HyperParameters object at 0x7f2566f13430>
    mesh = Mesh(device_ids=array([[[0, 2, 1, 3]]]), axis_names=('data', 'fsdp', 'tensor'))
)

In [62]:

  import checkpointing
  from jax import random
  import optax

# initialize the model
  checkpoint_manager = checkpointing.create_orbax_checkpoint_manager(config.checkpoint_dir,
                                                                     config.enable_checkpointing,
                                                                     config.async_checkpointing,
                                                                     config.save_period)
  rng = random.PRNGKey(0)


  tx = optax.adamw(
    max_utils.create_learning_rate_schedule(config)
  ) # TODO: we need an optax.GradientTransformation to form a TrainState, but we don't use it when decoding


  _, sp_tokenizer = create_data_iterator_with_tokenizer(config, mesh)

  state, state_mesh_annotations = max_utils.setup_initial_state(model, tx, config, rng, mesh, checkpoint_manager)

Creating checkpoint manager...
Checkpoint manager created!
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer
restoring state from gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints step 3399


In [72]:
import jax.numpy as jnp
jax.tree_util.tree_map(jnp.shape, state.params)

{'decoder': {'decoder': {'mlp': {'wi': {'kernel': (2560, 16, 8192)},
    'wo': {'kernel': (8192, 16, 2560)}},
   'pre_self_attention_layer_norm': {'scale': (2560, 16)},
   'self_attention': {'key': {'kernel': (2560, 16, 8, 256)},
    'key_layer_norm': {'scale': (256, 16)},
    'out': {'kernel': (8, 16, 256, 2560)},
    'query': {'kernel': (2560, 16, 8, 256)},
    'query_layer_norm': {'scale': (256, 16)},
    'value': {'kernel': (2560, 16, 8, 256)}}},
  'decoder_norm': {'scale': (2560,)}},
 'token_embedder': {'embedding': (32768, 2560)}}

In [73]:
state.params

{'decoder': {'decoder': {'mlp': {'wi': {'kernel': Array([[[ 8.98968522e-03,  8.28016270e-03,  3.67044657e-03, ...,
              -9.56583954e-03,  7.05189863e-03,  3.99113558e-02],
             [-3.82190198e-02, -4.10426892e-02,  3.00368369e-02, ...,
              -1.61877484e-03, -4.99821827e-02, -2.80014295e-02],
             [-1.86319798e-02,  1.96204167e-02,  1.54114952e-02, ...,
              -4.62821610e-02, -6.79477584e-03, -3.46121565e-02],
             ...,
             [-1.13918995e-02,  6.77212775e-02,  2.15508156e-02, ...,
              -1.04808928e-02, -2.07281522e-02, -3.89581081e-04],
             [ 4.06782236e-03, -7.35499011e-03,  1.00202775e-02, ...,
               4.26054560e-02,  2.14331038e-02,  7.41015247e-05],
             [ 7.60325743e-03,  3.06681748e-02, -3.42619047e-03, ...,
              -3.35110612e-02,  4.32843491e-02,  9.57179815e-03]],
     
            [[-2.82954169e-03,  3.28228585e-02,  9.01225582e-03, ...,
              -1.07359672e-02,  7.04857986e-

In [26]:
# Try out tokenization / detokenization
print(f"Loaded Tokenizer with vocab size of {sp_tokenizer.vocab_size()}")

sp_tokenizer.vocab_size()

tokenized = sp_tokenizer.tokenize("Hello world")

print(sp_tokenizer.id_to_string(1))
print(f"Tokenized: {tokenized}")

for token in tokenized:
    print(f"Token {token} is str: {sp_tokenizer.id_to_string(token)}")

detokenized = sp_tokenizer.detokenize(tokenized)
print(detokenized)
#for i in range(10):
#  print(f"Token {i} is: '{tf.print(sp_tokenizer.id_to_string(i)}'")

Loaded Tokenizer with vocab size of 32768
tf.Tensor(b'<s>', shape=(), dtype=string)
Tokenized: [3869  190    2]
Token 3869 is str: b'\xe2\x96\x81Hello'
Token 190 is str: b'\xe2\x96\x81world'
Token 2 is str: b'</s>'
tf.Tensor(b'Hello world', shape=(), dtype=string)


In [35]:
print(sp_tokenizer.detokenize([190]))

tf.Tensor(b'world', shape=(), dtype=string)


In [45]:
# try out train dataset
x = next(iter(train_ds))
print(x)
x['inputs'] == x["targets"]

{'content-length': <tf.Tensor: shape=(), dtype=string, numpy=b'3171'>, 'content-type': <tf.Tensor: shape=(), dtype=string, numpy=b'text/plain'>, 'timestamp': <tf.Tensor: shape=(), dtype=string, numpy=b'2019-04-20T22:24:20Z'>, 'url': <tf.Tensor: shape=(), dtype=string, numpy=b'http://madeinusaforever.com/o2afshba34oz.html'>, 'inputs': <tf.Tensor: shape=(), dtype=string, numpy=b"* Our award-winning O2XYGEN After Shave Balm.\n* Contains Beta Hydroxy for skin maintenance.\n* Not a burning alcohol product, but true skincare.\n* Matches O2XYGEN Eau de Toilette. Citrus fragrance botanical.\nO2XYGEN Men's After Shave Balm has a safe level of Beta Hydroxy Acid to clean up dead skin cells and maintain healthy skin.\nThe scent of O2XYGEN is fresh and clean, a favorite used everyday for skincare and fragrance combined!\nNo one adds BHA to their after-shave balm except California North.\nFollow with O2XYGEN EDT across the mantle.\nA perfect complement to O2XYGEN men's eau de toilette.\nFormulated w

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [44]:
x


{'content-length': <tf.Tensor: shape=(), dtype=string, numpy=b'1319'>,
 'content-type': <tf.Tensor: shape=(), dtype=string, numpy=b'text/plain'>,
 'timestamp': <tf.Tensor: shape=(), dtype=string, numpy=b'2019-04-22T14:21:12Z'>,
 'url': <tf.Tensor: shape=(), dtype=string, numpy=b'https://balance.ventures/p/balance-daily/entrepreneur/lifestyle/never-stop-challenging-best-way-add-value/'>,
 'inputs': <tf.Tensor: shape=(), dtype=string, numpy=b'There are two pieces of advice that I would give any young person who wants to make an impact.\nWhat career and life advice would you give to your 18-year-old self? originally appeared on Quora: the place to gain and share knowledge, empowering people to learn from others and better understand the world.'>,
 'targets': <tf.Tensor: shape=(), dtype=string, numpy=b'There are two pieces of advice that I would give any young person who wants to make an impact.\nWhat career and life advice would you give to your 18-year-old self? originally appeared on Qu

In [50]:
x = train_iter()

In [51]:
x

{'inputs': Array([[    0,  2211,   641, ...,     0,     0,     0],
        [    0, 11539,  2878, ...,     0,     0,     0],
        [    0,    42,   215, ...,     0,     0,     0],
        ...,
        [    0,    72,  7186, ...,     0,     0,     0],
        [    0, 10032,    47, ...,     0,     0,     0],
        [    0,  6105,    16, ...,     0,     0,     0]], dtype=int32),
 'inputs_position': Array([[0, 1, 2, ..., 0, 0, 0],
        [0, 1, 2, ..., 0, 0, 0],
        [0, 1, 2, ..., 0, 0, 0],
        ...,
        [0, 1, 2, ..., 0, 0, 0],
        [0, 1, 2, ..., 0, 0, 0],
        [0, 1, 2, ..., 0, 0, 0]], dtype=int32),
 'inputs_segmentation': Array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32),
 'targets': Array([[ 2211,   641,     8, ...,     0,     0,     0],
        [11539,  2878,  5807, ...,     0,     0,     0],
 

In [4]:
import decode
import os
import jax

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
decode.decode_loop(config) 
jax.distributed.shutdown()

2023-11-15 21:38:09.268847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/patflick/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Initialized persistent compilation cache at /home/patflick/jax_cache


Creating checkpoint manager...
Checkpoint manager created!
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer
restoring state from gs://patflick-maxtext-lingvo/patemotter-1b/checkpoints step 3399


I0000 00:00:1700084294.985353  311833 gcs_resource.cc:99] Using default AdmissionQueue with limit 32
I0000 00:00:1700084294.988637  314440 google_auth_provider.cc:179] Running on GCE, using service account 903354779218-compute@developer.gserviceaccount.com


Decoding #0 (num tokens 50):
	What can you tell me about the performance of TPUs? how much of the TSPysysysysysysyssyssyssssysssss
Decoding #1 (num tokens 50):
	What can you tell me about the performance of TPUs? is not on my a . I don's , I's abt i's that i can i can's .m not bet'
Decoding #2 (num tokens 50):
	What can you tell me about the performance of TPUs? i'ss in .s in the ...i.i.. i's.i.i.i.e.i.i.i
Decoding #3 (num tokens 50):
	What can you tell me about the performance of TPUs? I's and why? I have abt ab2?t the other ab? I's? bs?b?b?b?b?
Decoding #4 (num tokens 50):
	What can you tell me about the performance of TPUs? I have a problem with the sbd to the ssb and i use the dcvm i need to the srpf and sb and ac
Decoding #5 (num tokens 50):
	What can you tell me about the performance of TPUs? i can you can you'ss and i's? i's? i bet i's? i's? i's? i's and
Decoding #6 (num tokens 50):
	What can you tell me about the performance of TPUs? and what is it? I should i can i want to get

KeyboardInterrupt: 

In [61]:
# 
import jax
jax.distributed.shutdown()